# Long-Form Alignment Test

This notebook tests the complete long-form alignment pipeline using **Meta's Q1 2025 Earnings Call** (~1 hour audio, ~9K words).

**Pipeline:**
1. Audio Frontend: Load, resample, segment
2. Text Frontend: Load PDF, normalize, tokenize
3. Labeling Utils: MMS-FA model inference
4. Alignment: WFST-based flexible alignment
5. Stitching: LIS-based segment concatenation
6. Visualization: Audacity labels, Gentle HTML, audio preview

**Tests:**
1. Setup and imports
2. Load audio (Meta earnings call)
3. Load text (PDF transcript)
4. Run alignment pipeline (step-by-step, Tutorial.py pattern)
4B. **High-level API test** (`align_long_audio()` - recommended for users)
5. Inspect results
6. Audacity label export
7. Gentle HTML visualization
8. Random segment listening test
9. Word-by-word listening test

## Setup

In [ ]:
# =============================================================================
# Install Dependencies (auto-detect k2 version)
# =============================================================================

import subprocess
import sys

def install_k2_if_needed():
    """Check if k2 is available, if not, install the correct version."""
    try:
        import k2
        print(f"k2 already installed:")
        ! pip show k2
        return True
    except ImportError:
        pass
    
    # Get system info
    import torch
    torch_version = torch.__version__.split('+')[0]  # e.g., "2.5.0"
    torch_major_minor = '.'.join(torch_version.split('.')[:2])  # e.g., "2.5"
    cuda_available = torch.cuda.is_available()
    cuda_version = torch.version.cuda if cuda_available else None
    
    print(f"PyTorch: {torch_version}")
    print(f"CUDA available: {cuda_available}")
    if cuda_version:
        print(f"CUDA version: {cuda_version}")
    
    # Determine which k2 to install
    if cuda_available and cuda_version:
        # GPU version
        cuda_major_minor = '.'.join(cuda_version.split('.')[:2])  # e.g., "12.4"
        index_url = "https://k2-fsa.github.io/k2/cuda.html"
        print(f"\nLooking for k2 with CUDA {cuda_major_minor} and PyTorch {torch_major_minor}...")
        
        # Try to find matching version from the index
        # Common patterns: k2==1.24.4.dev20251030+cuda12.4.torch2.5.0
        try:
            import urllib.request
            with urllib.request.urlopen(index_url, timeout=10) as response:
                html = response.read().decode('utf-8')
            
            # Parse available versions
            import re
            # Match pattern like: k2-1.24.4.dev20251030+cuda12.4.torch2.5.0
            pattern = rf'k2-[\d.]+dev\d+\+cuda{re.escape(cuda_major_minor)}\.torch{re.escape(torch_major_minor)}\.\d+'
            matches = re.findall(pattern, html)
            
            if matches:
                # Get the latest version (last match usually)
                latest = matches[-1].replace('k2-', 'k2==').replace('+', '%2B')
                # Convert back for pip
                pkg_name = matches[-1].replace('k2-', 'k2==')
                print(f"Found: {pkg_name}")
                cmd = f"pip install {pkg_name} -f {index_url}"
            else:
                print(f"No exact match found for CUDA {cuda_major_minor} + PyTorch {torch_major_minor}")
                print("Trying generic GPU install...")
                cmd = f"pip install k2 -f {index_url}"
        except Exception as e:
            print(f"Could not fetch index: {e}")
            cmd = f"pip install k2 -f {index_url}"
    else:
        # CPU version
        index_url = "https://k2-fsa.github.io/k2/cpu.html"
        print(f"\nLooking for k2 CPU version for PyTorch {torch_major_minor}...")
        
        try:
            import urllib.request
            with urllib.request.urlopen(index_url, timeout=10) as response:
                html = response.read().decode('utf-8')
            
            import re
            pattern = rf'k2-[\d.]+dev\d+\+cpu\.torch{re.escape(torch_major_minor)}\.\d+'
            matches = re.findall(pattern, html)
            
            if matches:
                pkg_name = matches[-1].replace('k2-', 'k2==')
                print(f"Found: {pkg_name}")
                cmd = f"pip install {pkg_name} --no-deps -f {index_url}"
            else:
                print(f"No exact match found for PyTorch {torch_major_minor}")
                cmd = f"pip install k2 --no-deps -f {index_url}"
        except Exception as e:
            print(f"Could not fetch index: {e}")
            cmd = f"pip install k2 --no-deps -f {index_url}"
    
    print(f"\nInstalling: {cmd}")
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print("k2 installed successfully!")
        return True
    else:
        print(f"Installation failed: {result.stderr}")
        return False

def install_other_deps():
    """Install other required dependencies."""
    deps = [
        "pytorch-lightning",
        "cmudict",
        "g2p_en",
        "pydub",
        "pypdf",
        "git+https://github.com/huangruizhe/lis.git",
    ]
    for dep in deps:
        try:
            subprocess.run(f"pip install -q {dep}", shell=True, check=True)
        except:
            print(f"Warning: Failed to install {dep}")

# Run installation
install_k2_if_needed()
install_other_deps()
print("\nDependency installation complete.")

In [ ]:
# =============================================================================
# Setup: Clone Repository and Configure Imports
# =============================================================================

import sys
import os
from pathlib import Path

# ===== CONFIGURATION =====
GITHUB_REPO = "https://github.com/huangruizhe/torchaudio_aligner.git"
BRANCH = "dev"
# =========================

test_results = {}

def setup_imports():
    IN_COLAB = 'google.colab' in sys.modules
    
    if IN_COLAB:
        repo_path = '/content/torchaudio_aligner'
        src_path = f'{repo_path}/src'
        examples_path = '/content/examples'
        
        if not os.path.exists(repo_path):
            print(f"Cloning repository (branch: {BRANCH})...")
            os.system(f'git clone -b {BRANCH} {GITHUB_REPO} {repo_path}')
        else:
            print(f"Updating repository (branch: {BRANCH})...")
            os.system(f'cd {repo_path} && git fetch origin && git checkout {BRANCH} && git pull origin {BRANCH}')
        
        # Download Meta earnings call if not present
        if not os.path.exists(examples_path):
            os.makedirs(examples_path)
        
        if not os.path.exists(f'{examples_path}/4780182.mp3'):
            print("Downloading Meta Q1 2025 Earnings Call audio...")
            os.system(f'wget -q https://static.seekingalpha.com/cdn/s3/transcripts_audio/4780182.mp3 -O {examples_path}/4780182.mp3')
        
        if not os.path.exists(f'{examples_path}/META-Q1-2025-Earnings-Call-Transcript-1.pdf'):
            print("Downloading Meta Q1 2025 Earnings Call transcript...")
            os.system(f'wget -q https://s21.q4cdn.com/399680738/files/doc_financials/2025/q1/Transcripts/META-Q1-2025-Earnings-Call-Transcript-1.pdf -O {examples_path}/META-Q1-2025-Earnings-Call-Transcript-1.pdf')
    else:
        possible_paths = [
            Path(".").absolute().parent / "src",
            Path(".").absolute() / "src",
        ]
        src_path = None
        for p in possible_paths:
            if p.exists() and (p / "alignment").exists():
                src_path = str(p.absolute())
                break
        if src_path is None:
            raise FileNotFoundError("src directory not found")
        
        # Examples in parent directory
        examples_path = str(Path(src_path).parent.parent / "examples")
        print(f"Running locally from: {src_path}")
    
    if src_path not in sys.path:
        sys.path.insert(0, src_path)
    
    return src_path, examples_path

src_path, examples_path = setup_imports()

import torch
import torchaudio
import logging
logging.basicConfig(level=logging.INFO)

print()
print("=" * 60)
print(f"PyTorch: {torch.__version__}")
print(f"TorchAudio: {torchaudio.__version__}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")
print(f"Examples: {examples_path}")
print("=" * 60)

In [ ]:
# Check dependencies
print("Checking dependencies...")

K2_AVAILABLE = False
LIS_AVAILABLE = False

try:
    import k2
    K2_AVAILABLE = True
    print("k2: available")
except ImportError:
    print("k2: NOT AVAILABLE - install with pip")

try:
    import lis
    LIS_AVAILABLE = True
    print("lis: available")
except ImportError:
    print("lis: NOT AVAILABLE - pip install git+https://github.com/huangruizhe/lis.git")

try:
    from pypdf import PdfReader
    print("pypdf: available")
except ImportError:
    print("pypdf: NOT AVAILABLE - pip install pypdf")

try:
    from pydub import AudioSegment
    print("pydub: available")
except ImportError:
    print("pydub: NOT AVAILABLE - pip install pydub")

In [ ]:
!pip install pytorch-lightning
!pip install cmudict g2p_en
!pip install pydub
!pip install git+https://github.com/huangruizhe/lis.git
!pip install torchcodec

## Test 1: Module Imports

In [ ]:
print("=" * 60)
print("Test 1: Module Imports")
print("=" * 60)

try:
    # Core modules
    from audio_frontend import AudioFrontend, segment_audio, segment_waveform
    from text_frontend import load_text_from_pdf, normalize_for_mms, CharTokenizer, create_tokenizer_from_labels
    from labeling_utils import load_model
    
    # Alignment - following Tutorial.py pattern
    from alignment import AlignmentResult, AlignedWord, AlignmentConfig
    from alignment.wfst.factor_transducer import make_factor_transducer_word_level_index_with_skip
    from alignment.wfst.k2_utils import align_segments, concat_alignments, get_final_word_alignment
    from alignment.wfst.lis_utils import compute_lis, remove_outliers, find_unaligned_regions
    from alignment.base import AlignedToken
    
    print("All modules imported successfully!")
    test_results["Test 1"] = "PASSED"
    print("\nTest 1 PASSED")
except Exception as e:
    test_results["Test 1"] = "FAILED"
    print(f"Test 1 FAILED: {e}")
    import traceback; traceback.print_exc()

## Test 2: Load Audio

In [ ]:
print("=" * 60)
print("Test 2: Load Audio (Meta Q1 2025 Earnings Call)")
print("=" * 60)

try:
    AUDIO_FILE = f"{examples_path}/4780182.mp3"
    
    print(f"\nLoading: {AUDIO_FILE}")
    
    # Load and preprocess
    waveform, orig_sr = torchaudio.load(AUDIO_FILE)
    print(f"  Original: {waveform.shape}, {orig_sr}Hz")
    
    # Resample to 16kHz
    if orig_sr != 16000:
        waveform = torchaudio.functional.resample(waveform, orig_sr, 16000)
    sr = 16000
    
    # Convert to mono
    if waveform.size(0) > 1:
        waveform = waveform.mean(0, keepdim=True)
    
    duration_sec = waveform.size(1) / sr
    duration_min = duration_sec / 60
    
    print(f"  Processed: {waveform.shape}, {sr}Hz")
    print(f"  Duration: {duration_sec:.1f}s ({duration_min:.1f} minutes)")
    
    # Segment
    from audio_frontend import segment_waveform
    segmentation = segment_waveform(
        waveform.squeeze(0),
        sample_rate=sr,
        segment_size=15.0,
        overlap=2.0,
    )
    print(f"  Segments: {segmentation.num_segments}")
    
    test_results["Test 2"] = "PASSED"
    print("\nTest 2 PASSED")
except Exception as e:
    test_results["Test 2"] = "FAILED"
    print(f"Test 2 FAILED: {e}")
    import traceback; traceback.print_exc()

## Test 3: Load Text

In [ ]:
print("=" * 60)
print("Test 3: Load Text (Meta Q1 2025 Earnings Call Transcript)")
print("=" * 60)

try:
    PDF_FILE = f"{examples_path}/META-Q1-2025-Earnings-Call-Transcript-1.pdf"
    
    print(f"\nLoading: {PDF_FILE}")
    
    # Load PDF
    text = load_text_from_pdf(PDF_FILE)
    print(f"  Raw text: {len(text)} characters")
    print(f"  Raw words: {len(text.split())}")
    
    # Preview
    print(f"\n  Preview (first 500 chars):")
    print(f"  {text[:500]}...")
    
    # Normalize for MMS
    text_normalized = normalize_for_mms(text, expand_numbers=True)
    text_words = text_normalized.split()
    print(f"\n  Normalized words: {len(text_words)}")
    print(f"  Preview: {' '.join(text_words[:20])}...")
    
    test_results["Test 3"] = "PASSED"
    print("\nTest 3 PASSED")
except Exception as e:
    test_results["Test 3"] = "FAILED"
    print(f"Test 3 FAILED: {e}")
    import traceback; traceback.print_exc()

## Test 4: Run Alignment Pipeline

In [ ]:
print("=" * 60)
print("Test 4: Run Alignment Pipeline (following Tutorial.py)")
print("=" * 60)

if not K2_AVAILABLE or not LIS_AVAILABLE:
    test_results["Test 4"] = "SKIPPED"
    print("SKIPPED - k2 or lis not available")
else:
    try:
        from tqdm import tqdm
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Device: {device}")
        
        # Step 1: Load model
        print("\nStep 1: Loading MMS-FA model...")
        model = load_model("mms-fa")
        vocab = model.get_vocab_info()
        print(f"  Vocab size: {len(vocab.labels)}")
        
        # Step 2: Create tokenizer
        print("\nStep 2: Creating tokenizer...")
        tokenizer = create_tokenizer_from_labels(
            tuple(vocab.labels),
            blank_token=vocab.blank_token,
            unk_token=vocab.unk_token,
        )
        text_tokenized = tokenizer.encode(text_normalized)
        print(f"  Tokenized: {len(text_tokenized)} words")
        
        # Step 3: Build WFST
        print("\nStep 3: Building WFST decoding graph...")
        decoding_graph, word_index_sym_tab, token_sym_tab = \
            make_factor_transducer_word_level_index_with_skip(
                text_tokenized,
                blank_penalty=0,
                skip_penalty=-0.5,
                return_penalty=-18.0,
            )
        decoding_graph = decoding_graph.to(device)
        print(f"  Nodes: {decoding_graph.shape[0]}, Arcs: {decoding_graph.num_arcs}")
        
        # Step 4: Align segments (following Tutorial.py pattern)
        print("\nStep 4: Aligning segments...")
        batch_size = 32 if device.type == "cuda" else 4
        frame_duration = 0.02
        
        waveforms_batched, lengths = segmentation.get_waveforms_batched()
        offsets = segmentation.get_offsets_in_frames(frame_duration)
        
        alignment_results = []
        
        for i in tqdm(range(0, segmentation.num_segments, batch_size)):
            batch_waveforms = waveforms_batched[i:i+batch_size].to(device)
            batch_lengths = lengths[i:i+batch_size].to(device)
            batch_offsets = offsets[i:i+batch_size]
            
            with torch.inference_mode():
                emissions, emission_lengths = model.get_emissions(batch_waveforms, batch_lengths)
            
            # Use align_segments (following Tutorial.py pattern)
            batch_results = align_segments(
                emissions,
                decoding_graph,
                emission_lengths,
            )
            
            # Add frame offsets and word indices (following Tutorial.py)
            for aligned_tokens, offset in zip(batch_results, batch_offsets):
                offset_val = offset.item()
                for token in aligned_tokens:
                    token.timestamp += offset_val  # Absolute frame timestamp
                    if token.token_id == tokenizer.blk_id:
                        continue
                    if token.token_id in word_index_sym_tab:
                        token.attr["wid"] = word_index_sym_tab[token.token_id]
                    if token.token_id in token_sym_tab:
                        token.attr["tk"] = token_sym_tab[token.token_id]
            
            alignment_results.extend(batch_results)
            
            # Break early for CPU demo
            if device.type == "cpu" and i >= 8:
                print("  (CPU mode: stopping after 8 batches for demo)")
                break
        
        print(f"  Aligned {len(alignment_results)} segments")
        
        # Step 5: Concatenate alignments (following Tutorial.py pattern)
        print("\nStep 5: Concatenating alignments...")
        stitched_tokens = concat_alignments(alignment_results)
        print(f"  Stitched tokens: {len(stitched_tokens)}")
        
        # Step 6: Get final word alignment (following Tutorial.py pattern)
        print("\nStep 6: Building word alignment...")
        word_alignment = get_final_word_alignment(stitched_tokens, text_words)
        print(f"  Aligned words: {len(word_alignment)}")
        
        # Find unaligned regions
        aligned_indices = set(word_alignment.keys())
        if aligned_indices:
            rg_min, rg_max = min(aligned_indices), max(aligned_indices)
            unaligned_indices = find_unaligned_regions(rg_min, rg_max, aligned_indices)
            print(f"  Unaligned regions: {len(unaligned_indices)}")
        else:
            unaligned_indices = []
        
        test_results["Test 4"] = "PASSED"
        print("\nTest 4 PASSED")
        
    except Exception as e:
        test_results["Test 4"] = "FAILED"
        print(f"Test 4 FAILED: {e}")
        import traceback; traceback.print_exc()

In [ ]:
print("=" * 60)
print("Test 4B: High-Level API (align_long_audio)")
print("=" * 60)

if not K2_AVAILABLE or not LIS_AVAILABLE:
    test_results["Test 4B"] = "SKIPPED"
    print("SKIPPED - k2 or lis not available")
else:
    try:
        from api import align_long_audio, LongFormAlignmentResult
        
        print("\nUsing high-level API: align_long_audio()")
        print("This is the recommended way to use the library.\n")
        
        # Simple one-liner API call
        result = align_long_audio(
            audio=AUDIO_FILE,
            text=PDF_FILE,
            language="eng",
            verbose=True,
        )
        
        # Use convenience methods on result object
        print("\n" + "=" * 40)
        print("Result Summary:")
        print("=" * 40)
        print(result.summary())
        
        # Test save methods
        print("\nTesting result.save_audacity_labels()...")
        output_path = f"{examples_path}/meta_earnings_highlevel_labels.txt"
        result.save_audacity_labels(output_path)
        print(f"  Saved to: {output_path}")
        
        # Store for later tests
        word_alignment_highlevel = result.word_alignments
        
        test_results["Test 4B"] = "PASSED"
        print("\nTest 4B PASSED")
        
    except Exception as e:
        test_results["Test 4B"] = "FAILED"
        print(f"Test 4B FAILED: {e}")
        import traceback; traceback.print_exc()

## Test 5: Inspect Results

In [ ]:
print("=" * 60)
print("Test 5: Inspect Alignment Results")
print("=" * 60)

if "Test 4" not in test_results or test_results["Test 4"] != "PASSED":
    test_results["Test 5"] = "SKIPPED"
    print("SKIPPED - Test 4 did not pass")
else:
    try:
        print(f"\nAligned {len(word_alignment)} out of {len(text_words)} words")
        print(f"Coverage: {100*len(word_alignment)/len(text_words):.1f}%")
        
        # First 10 aligned words
        print("\nFirst 10 aligned words:")
        for i, (idx, word) in enumerate(sorted(word_alignment.items())[:10]):
            start_sec = word.start_time * frame_duration
            end_sec = word.end_time * frame_duration if word.end_time else start_sec + 0.3
            print(f"  [{idx:4d}] {word.word:15s} {start_sec:7.2f}s - {end_sec:7.2f}s")
        
        # Last 10 aligned words
        print("\nLast 10 aligned words:")
        for i, (idx, word) in enumerate(sorted(word_alignment.items())[-10:]):
            start_sec = word.start_time * frame_duration
            end_sec = word.end_time * frame_duration if word.end_time else start_sec + 0.3
            print(f"  [{idx:4d}] {word.word:15s} {start_sec:7.2f}s - {end_sec:7.2f}s")
        
        # Unaligned regions
        if unaligned_indices:
            print(f"\nUnaligned regions (showing first 5 of {len(unaligned_indices)}):")
            for s, e in unaligned_indices[:5]:
                if e - s > 0:
                    unaligned_text = " ".join(text_words[s:e+1][:5])
                    if e - s > 5:
                        unaligned_text += "..."
                    print(f"  [{s}, {e}]: {unaligned_text}")
        
        test_results["Test 5"] = "PASSED"
        print("\nTest 5 PASSED")
        
    except Exception as e:
        test_results["Test 5"] = "FAILED"
        print(f"Test 5 FAILED: {e}")

## Test 6: Audacity Label Export

In [ ]:
print("=" * 60)
print("Test 6: Audacity Label Export")
print("=" * 60)

if "Test 4" not in test_results or test_results["Test 4"] != "PASSED":
    test_results["Test 6"] = "SKIPPED"
    print("SKIPPED - Test 4 did not pass")
else:
    try:
        labels = get_audacity_labels(word_alignment, frame_duration=0.02)
        
        print("\nAudacity labels (first 10 lines):")
        for line in labels.split("\n")[:10]:
            print(f"  {line}")
        
        # Save to file
        output_path = f"{examples_path}/meta_earnings_labels.txt"
        save_audacity_labels(word_alignment, output_path, frame_duration=0.02)
        print(f"\nSaved to: {output_path}")
        print("\nTo use in Audacity:")
        print("  1. Open the audio file in Audacity")
        print("  2. File > Import > Labels")
        print(f"  3. Select {output_path}")
        
        test_results["Test 6"] = "PASSED"
        print("\nTest 6 PASSED")
        
    except Exception as e:
        test_results["Test 6"] = "FAILED"
        print(f"Test 6 FAILED: {e}")

## Test 7: Gentle HTML Visualization

In [ ]:
print("=" * 60)
print("Test 7: Gentle HTML Visualization")
print("=" * 60)

if "Test 4" not in test_results or test_results["Test 4"] != "PASSED":
    test_results["Test 7"] = "SKIPPED"
    print("SKIPPED - Test 4 did not pass")
else:
    try:
        # Save HTML
        html_path = f"{examples_path}/meta_earnings_visualization.html"
        save_gentle_html(
            word_alignment,
            text_normalized,
            html_path,
            audio_file=AUDIO_FILE,
            frame_duration=0.02,
            title="Meta Q1 2025 Earnings Call - Alignment Visualization",
        )
        
        print(f"\nSaved to: {html_path}")
        print("\nTo view:")
        print("  1. Download the HTML file")
        print("  2. Open in a web browser")
        print("  3. Click on words to play audio")
        
        # Show preview in notebook
        from IPython.display import HTML, display
        preview_html = get_gentle_visualization(
            word_alignment,
            text_normalized,
            frame_duration=0.02,
            i_word_end=200,  # Just first 200 words for preview
        )
        print("\nPreview (first 200 words):")
        display(HTML(preview_html))
        
        test_results["Test 7"] = "PASSED"
        print("\nTest 7 PASSED")
        
    except Exception as e:
        test_results["Test 7"] = "FAILED"
        print(f"Test 7 FAILED: {e}")

## Test 8: Random Segment Listening Test

In [ ]:
print("=" * 60)
print("Test 8: Random Segment Listening Test")
print("=" * 60)

if "Test 4" not in test_results or test_results["Test 4"] != "PASSED":
    test_results["Test 8"] = "SKIPPED"
    print("SKIPPED - Test 4 did not pass")
else:
    try:
        from IPython.display import display, Audio
        import random
        
        print("\nPlaying 3 random segments (50 words each):")
        print("=" * 60)
        
        for i in range(3):
            audio_widget, words, start_idx = preview_random_segment(
                waveform.squeeze(0),
                word_alignment,
                num_words=50,
                sample_rate=sr,
                frame_duration=0.02,
            )
            
            print(f"\nSegment {i+1} (starting at position {start_idx}):")
            if audio_widget:
                display(audio_widget)
            print("-" * 40)
        
        test_results["Test 8"] = "PASSED"
        print("\nTest 8 PASSED")
        
    except Exception as e:
        test_results["Test 8"] = "FAILED"
        print(f"Test 8 FAILED: {e}")
        import traceback; traceback.print_exc()

## Test 9: Word-by-Word Listening Test

In [ ]:
print("=" * 60)
print("Test 9: Word-by-Word Listening Test")
print("=" * 60)

if "Test 4" not in test_results or test_results["Test 4"] != "PASSED":
    test_results["Test 9"] = "SKIPPED"
    print("SKIPPED - Test 4 did not pass")
else:
    try:
        from IPython.display import display, Audio
        
        # Pick a random starting point
        import random
        sorted_items = sorted(word_alignment.items())
        start = random.randint(0, max(0, len(sorted_items) - 20))
        
        print(f"\nPlaying words {start} to {start+10}:")
        print("=" * 60)
        
        for i, (word_idx, word) in enumerate(sorted_items[start:start+10]):
            start_sec = word.start_time * 0.02
            end_sec = word.end_time * 0.02 if word.end_time else start_sec + 0.3
            
            print(f"\n[{word_idx}] '{word.word}' ({start_sec:.2f}s - {end_sec:.2f}s):")
            
            audio_widget = preview_word(
                waveform.squeeze(0),
                word_alignment,
                word_idx,
                sample_rate=sr,
                frame_duration=0.02,
            )
            if audio_widget:
                display(audio_widget)
        
        test_results["Test 9"] = "PASSED"
        print("\nTest 9 PASSED")
        
    except Exception as e:
        test_results["Test 9"] = "FAILED"
        print(f"Test 9 FAILED: {e}")
        import traceback; traceback.print_exc()

## Test Summary

In [ ]:
print("=" * 60)
print("TEST RESULTS SUMMARY")
print("=" * 60)

print()
for test_name, result in sorted(test_results.items(), key=lambda x: int(x[0].split()[1])):
    status = "PASSED" if result == "PASSED" else ("SKIPPED" if result == "SKIPPED" else "FAILED")
    icon = "[PASS]" if status == "PASSED" else ("[SKIP]" if status == "SKIPPED" else "[FAIL]")
    print(f"  {icon}  {test_name}")

passed = sum(1 for r in test_results.values() if r == "PASSED")
failed = sum(1 for r in test_results.values() if r == "FAILED")
skipped = sum(1 for r in test_results.values() if r == "SKIPPED")

print()
print(f"  Passed:  {passed}")
print(f"  Skipped: {skipped}")
print(f"  Failed:  {failed}")
print()

if failed == 0:
    print("All tests passed (or skipped due to missing dependencies)!")
else:
    print(f"{failed} test(s) failed")

## Interactive: Explore Alignment

Use the cells below to interactively explore the alignment results.

In [ ]:
# Play a specific segment by position
# Change the start_idx to explore different parts

if "Test 4" in test_results and test_results["Test 4"] == "PASSED":
    from IPython.display import display
    
    start_idx = 100  # Change this to explore different parts
    num_words = 30
    
    audio, words = preview_segment(
        waveform.squeeze(0),
        word_alignment,
        start_idx,
        num_words,
        sample_rate=sr,
        frame_duration=0.02,
    )
    
    if audio:
        display(audio)